In [1]:
import requests
from bs4 import BeautifulSoup as BS

import pandas as pd
pd.set_option('max_colwidth', 150)
pd.set_option('max_columns', 150)

import time
import pickle


In [6]:
def get_data(url):
    '''Get data and return json'''
    while True:
        try:
            r = requests.get(url)
            r_json = r.json()
            return r_json
        except: # [ConnectionError, MaxRetryError]:
            print('********************')
            print('TRAVOU NO GET DATA')
            print('********************')
            time.sleep(5)

def lista_cidades():
    '''
    Retorna a lista de cidades.
    Ex:
    https://listadeespera.saude.sc.gov.br/municipio/group/ibge/all
    '''
    url = 'https://listadeespera.saude.sc.gov.br/municipio/group/ibge/all'
    return get_data(url)

def lista_cnes(_ibge=0):
    '''
    Retorna a lista de centrais de regulação.
    Ex:
    https://listadeespera.saude.sc.gov.br/filas/responsavel/fila/byIbge/4211892    
    '''    
    if _ibge == 0:
        url = 'https://listadeespera.saude.sc.gov.br/filas/responsavel/fila'
    else:
        url = 'https://listadeespera.saude.sc.gov.br/filas/responsavel/fila/byIbge/{}'.format(_ibge)
    return [reg['_id'] for reg in get_data(url)]

def list_proc(cnes, mod, tp_fila=0):
    '''
    Pega os tipos de procedimentos de uma determinada central de regulação.

    :param cnes: int, corresponde ao byCnes = central de regulação
    :param mod: int, corresponde ao byModalidade = {0: 'consulta', 1: 'exame': 2:'cirurgia_eletiva',
    3: 'outros_procedimentos'}
    :param tp_fila: int, corresponde ao byTipoFila = {0: 'todos', 1: 'regulada', 2:'cronologica'}

    Ex:
    url = https://listadeespera.saude.sc.gov.br/filas/byCnes/9208305/fila/byModalidade/1/byTipoFila/0
    url = https://listadeespera.saude.sc.gov.br/filas/byCnes/7606907/fila/byModalidade/0/byTipoFila/0
    '''
    # não precisa separar por ibge, a lista de procedimentos é vinculada somente ao cnes
    url = 'https://listadeespera.saude.sc.gov.br/filas/byCnes/{}/fila/byModalidade/{}/byTipoFila/{}'.\
            format(cnes, mod, tp_fila)
    return get_data(url)

def wait_list(cnes, id_fila, tp_fila=0, ibge=1):
    '''
    Retorna a lista de espera.

    :param cnes: int, corresponde ao byCnes = central de regulação
    :param fila: int, corresponde ao fila = cod da resposta final
    :param ibge: int, não seo do que se trata

    Ex:
    https://listadeespera.saude.sc.gov.br/filas/byCnes/5089778/fila/3904003/usuarios/byTipoFila/0/byIbge/1
    https://listadeespera.saude.sc.gov.br/filas/byCnes/5089778/fila/1101000/usuarios/byTipoFila/0/byIbge/1
    https://listadeespera.saude.sc.gov.br/filas/byCnes/9513639/fila/0701141/usuarios/byTipoFila/0/byIbge/4200200
    '''
    url = 'https://listadeespera.saude.sc.gov.br/filas/byCnes/{}/fila/{}/usuarios/byTipoFila/{}/byIbge/{}'.\
            format(cnes, id_fila, tp_fila, ibge)
    return get_data(url)

In [3]:
%%time
time_sleep = 0
lista_cid = lista_cidades()
byModalidade = {0: 'consulta', 1: 'exame', 2:'cirurgia_eletiva', 
                3: 'outros_procedimentos'}
# Falta fazer o modalidade 0 => Consultas
modalidades = [3, 0, 1, 2]

# carrega o contador, caso não exista, cria um
try:
    contador = pickle.load(open('contador.p', 'rb'))
except:
    contador = []

for _mod in modalidades:
    # iterar as cidades
    for _cidade in lista_cid:
        _lista_cnes = lista_cnes(_cidade['ibge'])
        
        # iterar as centrais de regulação
        for _cnes in _lista_cnes:
            lista_df = []
            time.sleep(time_sleep)
            _lista_proc = list_proc(_cnes['CnesEstab'], _mod)

            # itera os procedimentos de cada central de regulação:
            for proc in _lista_proc:
                time.sleep(time_sleep)
                cont = (_mod, _cidade, _cnes, proc)
                if cont not in contadores:
                    fila = wait_list(_cnes['CnesEstab'], proc['_id'], ibge=_cidade['ibge'])
                    for r in fila:
                        lista_df.append(r)
                contador.append(cont)
                if len(contador) % 100 = 0:
                    pickle.dump(contador, open('contador.p', 'wb'))
                
            if lista_df != []:
                df = pd.DataFrame(lista_df)
                df['modalidade'] = byModalidade[_mod]
                df['municipio'] = _cidade['descricao']
                df.to_csv('produced_datasets/central_reg_{}_{}_{}.csv'.format(_cnes['CnesEstab'], 
                                                                       byModalidade[_mod], _cidade['ibge']))
                print('produced_datasets/central_reg_{}_{}_{}.csv'.format(_cnes['CnesEstab'], 
                                                                       byModalidade[_mod], _cidade['ibge']))


produced_datasets/central_reg_6442366_outros_procedimentos_4203253.csv
produced_datasets/central_reg_6013678_outros_procedimentos_4205555.csv
produced_datasets/central_reg_2300338_outros_procedimentos_4211892.csv
produced_datasets/central_reg_6013678_outros_procedimentos_4207577.csv
produced_datasets/central_reg_7994354_outros_procedimentos_4212239.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4211207.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4209805.csv
produced_datasets/central_reg_6013678_outros_procedimentos_4211801.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4200606.csv
produced_datasets/central_reg_6432670_outros_procedimentos_4219853.csv
produced_datasets/central_reg_6473652_outros_procedimentos_4205506.csv
produced_datasets/central_reg_6013678_outros_procedimentos_4205506.csv
produced_datasets/central_reg_6442366_outros_procedimentos_4211751.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4211751.csv
produc

produced_datasets/central_reg_6375642_outros_procedimentos_4215703.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4215703.csv
produced_datasets/central_reg_6442366_outros_procedimentos_4203402.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4203956.csv
produced_datasets/central_reg_7220758_outros_procedimentos_4208104.csv
produced_datasets/central_reg_6372732_outros_procedimentos_4206009.csv
produced_datasets/central_reg_2692260_outros_procedimentos_4202438.csv
produced_datasets/central_reg_6013678_outros_procedimentos_4200408.csv
produced_datasets/central_reg_2663090_consulta_4210209.csv
produced_datasets/central_reg_5993067_consulta_4202131.csv
produced_datasets/central_reg_6826636_consulta_4202859.csv
produced_datasets/central_reg_2672669_consulta_4211256.csv
produced_datasets/central_reg_6459706_consulta_4209854.csv
produced_datasets/central_reg_9474528_consulta_4219606.csv
produced_datasets/central_reg_6567339_consulta_4218251.csv
produced_datasets/c

produced_datasets/central_reg_6373372_exame_4204103.csv
produced_datasets/central_reg_7220758_exame_4202057.csv
produced_datasets/central_reg_9177647_exame_4218855.csv
produced_datasets/central_reg_7994354_exame_4212239.csv
produced_datasets/central_reg_6475922_exame_4202578.csv
produced_datasets/central_reg_5368677_exame_4210001.csv
produced_datasets/central_reg_6545998_exame_4217253.csv
produced_datasets/central_reg_2692260_exame_4217253.csv
produced_datasets/central_reg_7995059_exame_4211009.csv
produced_datasets/central_reg_6473652_exame_4205506.csv
produced_datasets/central_reg_6506909_exame_4203709.csv
produced_datasets/central_reg_9177787_exame_4207650.csv
produced_datasets/central_reg_6538045_exame_4219002.csv
produced_datasets/central_reg_6391737_exame_4212908.csv
produced_datasets/central_reg_6559328_exame_4215695.csv
produced_datasets/central_reg_9345965_exame_4216909.csv
produced_datasets/central_reg_5410665_exame_4203204.csv
produced_datasets/central_reg_9452079_exame_4210

produced_datasets/central_reg_9228632_cirurgia_eletiva_4202875.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4202875.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4202875.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4202875.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4210209.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4210209.csv
produced_datasets/central_reg_2663090_cirurgia_eletiva_4210209.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4210209.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4204756.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4204756.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4204756.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4202131.csv
produced_datasets/central_reg_5993067_cirurgia_eletiva_4202131.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4202131.csv
produced_datasets/central_reg_7220758_cirurgia_eletiva_4202859

produced_datasets/central_reg_6361315_cirurgia_eletiva_4211009.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4211009.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4211009.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4218905.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4218905.csv
produced_datasets/central_reg_6442366_cirurgia_eletiva_4218905.csv
produced_datasets/central_reg_2662906_cirurgia_eletiva_4218905.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4218905.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_6003044_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_5993067_cirurgia_eletiva_4205506.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4205506

...

produced_datasets/central_reg_6003044_cirurgia_eletiva_4208203.csv
produced_datasets/central_reg_5993067_cirurgia_eletiva_4208203.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4208203.csv
produced_datasets/central_reg_7937547_cirurgia_eletiva_4211603.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4211603.csv
produced_datasets/central_reg_6538045_cirurgia_eletiva_4211603.csv
produced_datasets/central_reg_6490212_cirurgia_eletiva_4211603.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4211603.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_9228632_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4207700.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4216057

produced_datasets/central_reg_6324169_cirurgia_eletiva_4214706.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4214706.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4201109.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4201109.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_9228632_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_6003044_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4217501.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4217204.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4217204.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4217204.csv
produced_datasets/central_reg_9208305_cirurgia_eletiva_4202107.csv
produced_datasets/central_reg_7606907_cirurgia_eletiva_4202107

produced_datasets/central_reg_6008763_cirurgia_eletiva_4207759.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4217907.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4217907.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4213005.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4213005.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4213005.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4215687.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4215687.csv
produced_datasets/central_reg_9228632_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_7947658_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_6442366_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_6003044_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_6013678_cirurgia_eletiva_4209003.csv
produced_datasets/central_reg_5993067_cirurgia_eletiva_4209003

produced_datasets/central_reg_7606907_cirurgia_eletiva_4211504.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4211504.csv
produced_datasets/central_reg_2418819_cirurgia_eletiva_4211504.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4211504.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4207809.csv
produced_datasets/central_reg_9228632_cirurgia_eletiva_4207809.csv
produced_datasets/central_reg_6003044_cirurgia_eletiva_4207809.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4207809.csv
********************
TRAVOU NO GET DATA
********************
produced_datasets/central_reg_6013678_cirurgia_eletiva_4207809.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4207809.csv
produced_datasets/central_reg_7859252_cirurgia_eletiva_4209151.csv
produced_datasets/central_reg_9007741_cirurgia_eletiva_4209151.csv
produced_datasets/central_reg_6324169_cirurgia_eletiva_4209151.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4209151.csv
p

produced_datasets/central_reg_6490212_cirurgia_eletiva_4206207.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4206207.csv
produced_datasets/central_reg_6361315_cirurgia_eletiva_4215752.csv
produced_datasets/central_reg_6008763_cirurgia_eletiva_4215752.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4215752.csv
produced_datasets/central_reg_7937547_cirurgia_eletiva_4204251.csv
produced_datasets/central_reg_7685661_cirurgia_eletiva_4204251.csv
produced_datasets/central_reg_6538045_cirurgia_eletiva_4204251.csv
produced_datasets/central_reg_6490212_cirurgia_eletiva_4204251.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4204251.csv
produced_datasets/central_reg_9208305_cirurgia_eletiva_4202453.csv
produced_datasets/central_reg_6003044_cirurgia_eletiva_4202453.csv
produced_datasets/central_reg_6324169_cirurgia_eletiva_4202453.csv
produced_datasets/central_reg_2692260_cirurgia_eletiva_4202453.csv
produced_datasets/central_reg_7859252_cirurgia_eletiva_4205100

Esse script aumentou de 40k acessos por 24h para 328k no site da filadeespera.
:-)

Quando separei por cidade caiu para menos da metade o numero de pacientes na fila.

Onde estão os outros? Não tem registros de cidade? Fiz alguma bobagem? Faltaram metade das cidades de SC na lista de cidades?

Por causa da diferença dos dados eu usei o script em que baixava separando por central de regulação. E como não há uma coluna separando o município, não fui capaz de analisar cada cidade separadamente.

In [13]:
%%time

# scrapping sem separar por cidade

time_sleep = 0
lista_cid = lista_cidades()
byModalidade = {0: 'consulta', 1: 'exame', 2:'cirurgia_eletiva', 
                3: 'outros_procedimentos'}

modalidades = [3, 0, 1, 2]

# carrega o contador, caso não exista, cria um
try:
    contador = pickle.load(open('contador.p', 'rb'))
except:
    contador = []

for _mod in modalidades:
#     # iterar as cidades
#     for _cidade in lista_cid:
        _lista_cnes = lista_cnes() #_cidade['ibge'])
        
        # iterar as centrais de regulação
        for _cnes in _lista_cnes:
            lista_df = []
            time.sleep(time_sleep)
            _lista_proc = list_proc(_cnes['CnesEstab'], _mod)

            # itera os procedimentos de cada central de regulação:
            for proc in _lista_proc:
                time.sleep(time_sleep)
                cont = (_mod, 
#                         _cidade, 
                        _cnes, 
                        proc)
                # verifica se já não foi copiado e, caso não tenha sido, copia.
                if cont not in contador:
                    fila = wait_list(_cnes['CnesEstab'], proc['_id']) #, ibge=_cidade['ibge'])
                    for r in fila:
                        lista_df.append(r)
                contador.append(cont)
                # salva o contador a cada X registros
                if len(contador) % 10 == 0:
                    pickle.dump(contador, open('contador.p', 'wb'))
            # caso haja algum procedimento para gravar, grave    
            if lista_df != []:
                df = pd.DataFrame(lista_df)
                df['modalidade'] = byModalidade[_mod]
#                 df['municipio'] = _cidade['descricao']
                df.to_csv('central_reg_{}_{}.csv'.format(_cnes['CnesEstab'],
                                                                       byModalidade[_mod])) #, _cidade['ibge']))
                print('central_reg_{}_{}.csv'.format(_cnes['CnesEstab'],
                                                                       byModalidade[_mod])) #, _cidade['ibge']))


central_reg_7859252_outros_procedimentos.csv
central_reg_9342389_outros_procedimentos.csv
********************
TRAVOU NO GET DATA
********************
********************
TRAVOU NO GET DATA
********************


KeyboardInterrupt: 